In [1]:
import pandas as pd

In [2]:
data_frame = pd.read_csv("fulldataset.csv")

In [3]:
useless_columns = list(data_frame.columns)
useless_columns = useless_columns[0:-4] + useless_columns[-2:]

In [4]:
data_frame = data_frame.drop(useless_columns,axis=1)

In [5]:
data_frame = data_frame.replace({"Good chair":1, "Bad chair":0})
gb_df = data_frame[~data_frame['Answer.category.label'].isin(["Inconclusive but likely bad","Inconclusive but likely good"])]

In [6]:
aggregation_functions = {'Answer.category.label': 'sum', 'Input.image_url': 'first'}

In [7]:
print(gb_df)

         Input.image_url Answer.category.label
0     all_views_2160.png                     1
1     all_views_2160.png                     1
3     all_views_2161.png                     1
4     all_views_2163.png                     0
6      all_views_123.png                     0
...                  ...                   ...
6177   all_views_421.png                     1
6178   all_views_421.png                     1
6179   all_views_423.png                     1
6181   all_views_424.png                     1
6182   all_views_424.png                     1

[5264 rows x 2 columns]


In [8]:
imgs = list(gb_df['Input.image_url'].unique())

In [9]:
final_gb_df = gb_df.copy()
for img in imgs:
    entries = final_gb_df.loc[final_gb_df['Input.image_url']==img]
    labels = list(entries.loc[: , 'Answer.category.label'])
    sum_labels = sum(labels)
    final_gb_df = final_gb_df.drop(entries.index)
    if(sum_labels == 0 or sum_labels == 2):
        label = sum([1 if sum_labels == 2 else 0])
        row = [[img,label]]
        col = ['Input.image_url', 'Answer.category.label']
        new_label = pd.DataFrame(row,columns=col)
        final_gb_df = final_gb_df.append(new_label,ignore_index = True)
print(final_gb_df)

         Input.image_url Answer.category.label
0     all_views_2160.png                     1
1     all_views_2163.png                     0
2      all_views_123.png                     0
3     all_views_2164.png                     0
4     all_views_1231.png                     0
...                  ...                   ...
2232   all_views_660.png                     1
2233   all_views_415.png                     1
2234   all_views_661.png                     1
2235   all_views_421.png                     1
2236   all_views_424.png                     1

[2237 rows x 2 columns]


In [10]:
#final_gb_df.to_csv("full_gb.csv", sep=',')

In [11]:
def prune(csv_file1, list_of_csv_files):
    """
    Input: csv_file1 = input csv file (e.g. full data set)
            list_of_csv_files = list of csv file locations containing data that we want removed from csv_file1

    Returns new df, which is df of csv_file1 without the entries in any of the csv files in list_of_csv_files
    """
    pd_frame1 = pd.read_csv(csv_file1)
    for file in list_of_csv_files:
        curr_df = pd.read_csv(file)
        df_imgs_in_curr_df = curr_df.iloc[:,0]
        imgs_in_curr_df = df_imgs_in_curr_df.values.tolist()
        pd_frame1 = pd_frame1[~pd_frame1['Input.image_url'].isin(imgs_in_curr_df)]
    return pd_frame1

list_csvs = ["train_data.csv","val_data.csv"]
pruned = prune("full_gb.csv",list_csvs)

In [12]:
#Checking if pruning worked
def prune_check(pruned_csv, list_of_csv_files):
    missed_ct = 0
    df = pruned_csv
    imgs_in_pruned_df = df['Input.image_url'].tolist()
    for file in list_of_csv_files:
        curr_df = pd.read_csv(file)
        df_imgs_in_curr_df = curr_df.iloc[:,0]
        imgs_in_curr_df = df_imgs_in_curr_df.values.tolist()
        missed_ct += sum([1 if curr_elem in imgs_in_pruned_df else 0 for curr_elem in imgs_in_curr_df ])
    return missed_ct
print(prune_check(pruned, list_csvs))

0


In [13]:
print(pruned)

      Unnamed: 0     Input.image_url  Answer.category.label
1              1  all_views_2163.png                      0
3              3  all_views_2164.png                      0
5              5  all_views_1232.png                      1
6              6  all_views_2167.png                      0
8              8  all_views_1235.png                      0
...          ...                 ...                    ...
2223        2223   all_views_408.png                      1
2224        2224    all_views_89.png                      0
2230        2230   all_views_412.png                      0
2234        2234   all_views_661.png                      1
2236        2236   all_views_424.png                      1

[687 rows x 3 columns]


In [16]:
pruned = pruned.drop(columns=['Unnamed: 0'])
print(pruned)

KeyError: "['Unnamed: 0'] not found in axis"

In [17]:
print(pruned)

         Input.image_url  Answer.category.label
1     all_views_2163.png                      0
3     all_views_2164.png                      0
5     all_views_1232.png                      1
6     all_views_2167.png                      0
8     all_views_1235.png                      0
...                  ...                    ...
2223   all_views_408.png                      1
2224    all_views_89.png                      0
2230   all_views_412.png                      0
2234   all_views_661.png                      1
2236   all_views_424.png                      1

[687 rows x 2 columns]


In [20]:
pruned.to_csv("testset_gb.csv", header=None, index=False,sep=',')